# Plot summary generator

In [ ]:
## adding directory to system path --- execute only once
#import sys,os
#os.chdir('C:\\Users\\Rohit Suresh\\Jupyter projects\\Plot summary generator')
#sys.path.append(os.getcwd())
#sys.path

In [3]:
# import necessary packages and libraries
from random import sample
import inspect
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
import importlib
import plot_summary_generator_utils
importlib.reload(plot_summary_generator_utils)
from plot_summary_generator_utils import *
import warnings
warnings.filterwarnings('ignore')
from gensim.models import Word2Vec, KeyedVectors

<IPython.core.display.Javascript object>

In [4]:
# see that all your functions from utils have been imported
all_functions = inspect.getmembers(plot_summary_generator_utils , inspect.isfunction)
for fn , _ in all_functions: print(str(fn))

IMDb
comic_plot
comic_plot_agg
comic_titles_finder
gcm
get_weight_matrix
get_wordnet_pos
imdb_id_list
lemma
make_comic_titles_list
pad_sequences
plot_aggregator_imdb
plot_aggregator_wiki
plot_puller_imdb
plot_puller_wiki
preprocess_corpus_text
sent_tokenize
titles_list
uReq
word_tokenize


# 1. Create text corpus
## 1.1. Obtain movie plot synopses

Plot synopsis for each Marvel comics based movie will be obtained from IMDb. We will use the 'IMDBPy' package to operate on IMDb pages. ###########cite

In [ ]:
# movies_filepath = 'corpus_movies_list.xlsx'
# imdb_ids = imdb_id_list(movies_filepath)
# imdb_ids[:5]

<code>plot_summary_generator_utils</code> contains a function <code>plot_aggregator_imdb</code> to collect movie synopses from IMDb and combine them into a single text file.<br>
<br>
<code>plot_aggregator_imdb</code> takes a list of IMDb IDs and a target file name as input. The function parses the webpage related to each movie (IMDB ID) in the input list and combines them into a .txt file with the name provided as target file.

In [ ]:
#%time plot_aggregator_imdb(imdb_ids,'all_movie_plots')

## 1.2. Obtain comic plot synopses
Plot synopsis for almost all comic books published by Marvel comics will be obtained from Marvel fandom website (https://marvel.fandom.com/wiki/Marvel_Database).  #############cite

We will use the BeautifulSoup package to operate on these webpages.

In [ ]:
# comic_filepath = 'comic_titles.csv' #file containing name of each comic book i.e. Title Volume Issue number
# c_t = pd.read_csv(comic_filepath,header=0,encoding = 'unicode_escape')['comic_title']
# comic_titles = list(c_t)

<code>plot_summary_generator_utils</code> contains a function <code>comic_plot_agg</code> to collect plot synopses from the Marvel fandom website and combine them into a single text file. <br>
<br>
<code>comic_plot_agg</code> takes a list of comic book titles and a target file name as input. The function parses the webpage related to each comic book in the input list and combines them into a .txt file with the name provided as target file.

In [ ]:
#%time comic_plot_agg(comic_titles,'all_comics_plots')

## 1.3. Combine movie and comic plots to create corpus
The corpus for this application is formed by combinig the plots from movies and the ones from the comics. This should give our generator model a good idea of what Marvel comics is all about!

In [ ]:
# with open('marvel_corpus.txt','w') as mf:
#     concat_plot = ''.join([open(f).read() for f in ['all_movies_plots.txt','all_comics_plots.txt']])
#     mf.write(concat_plot)

# 2. Train Word Embedding

## 2.1. Obtain text corpus

Import text from the .txt file containing our text corpus.

In [ ]:
with open('marvel_corpus.txt','r',encoding='utf-8') as f: raw_corpus = f.read()

## 2.2. Process text corpus

Now that we have a corpus to train our embedding on, it is important to clean it so that the embedding is trained without a hitch. Cleaning involves getting rid of punctuation, special symbols etc.<br>
<br>
Another important step to be taken before trainig a word embedding is tokenization i.e. separation of each word in each sentence into tokens. I have taken care of this in the <code>preprocess_corpus_text</code> helper function. Let's evoke this function to process our text corpus.

In [ ]:
#corpus_tokens = preprocess_corpus_text(raw_corpus,lemmatize = False,remove_stopwords = False)

## 2.3. Train word embedding
With the tokens we've kust created, we can now train a word embedding.<br>
<br>
We'll train a word embedding using the <code>Word2Vec</code> method present in the <code>gensim</code> package.<br>
<br>
We'll use the following settings for our embedding:
- size = 300 (Dimensions of each embedding)
- min_count = 1
- sg = 1 (Skip gram is being used)
- iter = 500 (Number of iterations)
- negative = 10 (Negative sampling)

In [ ]:
#import necessary libraries
# import gensim
# from gensim.models import Word2Vec, KeyedVectors

In [ ]:
# size = 300
# min_count = 1
# sg = 1
# num_iterations = 500
# negative = 10
# workers = 4
# marvel_embedding = Word2Vec(corpus_tokens, size = size, min_count = min_count, sg = sg, iter = num_iterations, negative=negative, workers = workers)

In [ ]:
# # save word embedding
# marvel_embedding.save('marvel_word_embedding')

# 3. Text generator

## 3.1. Pre-trained word embedding
Since we have already trained a word embedding on Marvel text, we will use that embedding in our model.

In [ ]:
# marvel_model_embedding = Word2Vec.load('marvel_word_embedding')
# print(marvel_model_embedding)

The Keras model that we'll use in this task uses an <code>Embedding</code> layer that can use pre-trained word embeddings. But this embedding layer requires the complete embedding matrix. The gensim method <code>wv.syn0</code> can be used to get the embedding matrix.

In [ ]:
# embedding_mtx = marvel_model_embedding.wv.syn0

## 3.2. Prepare training data
Our model will train on the corpus that we already have. We can also try shortening the corpus text to provide the model with limited amount of infoormation.<br>
<br>
But first, we need to form training sequences on which our model will train.<br>
<br>
An example of a training sequences is shown below:<br>
<br>
**Text:** 'a convoy of military humvees drives across the afghanistan desert in one of them is billionaire weapons developer tony stark'
- *Sequence 1:* 'a convoy of military humvees drives across the afghanistan desert in one of them is billionaire'
- *Sequence 2:* 'convoy of military humvees drives across the afghanistan desert in one of them is billionaire weapons'
- *Sequence 3:* 'of military humvees drives across the afghanistan desert in one of them is billionaire weapons developer'
- *Sequence 4:* 'military humvees drives across the afghanistan desert in one of them is billionaire weapons developer tony'
- *Sequence 5:* 'humvees drives across the afghanistan desert in one of them is billionaire weapons developer tony stark'
<br>
<br>
In the above 16 word sequences, the first 15 words will be the input - X, and the 16th word will be the output y.<br>
To achieve this format for our data, we will repurpose the tokens that we had previously used to train our word embedding.

In [ ]:
# # import necessary libraries
# from keras.preprocessing.text import Tokenizer

In [ ]:
# # Convert list of tokenized sentences into an array of word tokens
# token_array = []
# for sen in corpus_tokens:
#     for word in sen:
#         token_array.append(word)
# token_array

In [ ]:
# # Single word prediction
# # Form 16 word text sequences from array of word tokens
# length = 15 + 1
# sequences = list()
# for i in range(length, len(token_array)):
#     # select sequence of tokens
#     seq = token_array[i-length:i]
#     # convert into a line
#     line = ' '.join(seq)
#     # store
#     sequences.append(line)
# print('Total Sequences: %d' % len(sequences))

In [ ]:
# # Convert word tokens into numerical index tokens
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(sequences)
# tok_seq = tokenizer.texts_to_sequences(sequences)

In [ ]:
# # Form X and y combination from text sequences
# # separate into input and output
# tok_seq = np.array(tok_seq)
# X, y = tok_seq[:,:-1], tok_seq[:,-1]
# # One-hot encode y
# y = KU.to_categorical(y, num_classes=vocab_size)

In [ ]:
# # This information will be used later on while creating the Keras model
# vocab_size = len(tokenizer.word_index) + 1
# seq_length = X.shape[1]

## 3.3. Create Keras model

## Test gen

In [5]:
from keras.preprocessing.text import Tokenizer
import keras.utils as KU
import gensim
from itertools import chain
from pickle import dump
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding, LSTM, Bidirectional, Dropout, BatchNormalization, Flatten, Activation
from random import randint
import keras.backend as KB
from keras.initializers import Constant
from keras.callbacks import EarlyStopping
from keras_self_attention import SeqSelfAttention

In [7]:
with open('marvel_corpus.txt','r',encoding='utf-8') as f: text = f.read()

In [10]:
tokens = preprocess_corpus_text(text[:10000],lemmatize = False,remove_stopwords = False)

In [11]:
tokens

[['a',
  'convoy',
  'of',
  'military',
  'humvees',
  'drives',
  'across',
  'the',
  'afghanistan',
  'desert'],
 ['in',
  'one',
  'of',
  'them',
  'is',
  'billionaire',
  'weapons',
  'developer',
  'tony',
  'stark',
  'robert',
  'downey',
  'jr',
  'riding',
  'with',
  'soldiers',
  'on',
  'duty'],
 ['he',
  'is',
  'joking',
  'with',
  'some',
  'members',
  'of',
  'a',
  'convoy',
  'who',
  'seem',
  'to',
  'be',
  'genuinely',
  'amused',
  'by',
  'his',
  'persona',
  'and',
  'his',
  'flamboyant',
  'public',
  'image'],
 ['suddenly', 'the', 'convoy', 'is', 'ambushed', 'by', 'unseen', 'gunmen'],
 ['the',
  'soldiers',
  'fight',
  'to',
  'defend',
  'themselves',
  'but',
  'are',
  'quickly',
  'killed'],
 ['tony',
  'flees',
  'the',
  'humvee',
  'taking',
  'refuge',
  'behind',
  'a',
  'large',
  'rock'],
 ['moments',
  'later',
  'a',
  'missile',
  'lands',
  'nearby',
  'and',
  'explodes',
  'but',
  'not',
  'before',
  'tony',
  'sees',
  'the',
  '

In [12]:
%%notify
%time gen_embed = Word2Vec(tokens, size = 100, min_count = 1, sg = 1, iter=1, negative=10)

Wall time: 592 ms


<IPython.core.display.Javascript object>

In [13]:
gen_embed.save('expt_embed1')

In [14]:
embed = Word2Vec.load('expt_embed1')

In [15]:
print(embed)

Word2Vec(vocab=710, size=100, alpha=0.025)


In [16]:
embed_mtx = embed.wv.syn0 #get_weight_matrix(embed)

In [17]:
vocab_size = len(embed.wv.vocab)

In [18]:
embed_size = embed.vector_size

In [19]:
token_array = []
for sen in tokens:
    for word in sen:
        token_array.append(word)
token_array

['a',
 'convoy',
 'of',
 'military',
 'humvees',
 'drives',
 'across',
 'the',
 'afghanistan',
 'desert',
 'in',
 'one',
 'of',
 'them',
 'is',
 'billionaire',
 'weapons',
 'developer',
 'tony',
 'stark',
 'robert',
 'downey',
 'jr',
 'riding',
 'with',
 'soldiers',
 'on',
 'duty',
 'he',
 'is',
 'joking',
 'with',
 'some',
 'members',
 'of',
 'a',
 'convoy',
 'who',
 'seem',
 'to',
 'be',
 'genuinely',
 'amused',
 'by',
 'his',
 'persona',
 'and',
 'his',
 'flamboyant',
 'public',
 'image',
 'suddenly',
 'the',
 'convoy',
 'is',
 'ambushed',
 'by',
 'unseen',
 'gunmen',
 'the',
 'soldiers',
 'fight',
 'to',
 'defend',
 'themselves',
 'but',
 'are',
 'quickly',
 'killed',
 'tony',
 'flees',
 'the',
 'humvee',
 'taking',
 'refuge',
 'behind',
 'a',
 'large',
 'rock',
 'moments',
 'later',
 'a',
 'missile',
 'lands',
 'nearby',
 'and',
 'explodes',
 'but',
 'not',
 'before',
 'tony',
 'sees',
 'the',
 'stark',
 'industries',
 'logo',
 'painted',
 'on',
 'it',
 'a',
 'small',
 'amount',
 

In [20]:
#seq2seq approach:
ip_len = 15
op_len = 1
length = ip_len + op_len
sequences = list()
for i in range(length, len(token_array)):
    # select sequence of tokens
    seq = token_array[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 1693


In [21]:
sequences[-1]

'a test flight and he is thrilled by the suits functionality tony pushes the limit for'

In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
tok_seq = tokenizer.texts_to_sequences(sequences)

In [23]:
vocab_size = len(tokenizer.word_index) + 1

In [24]:
# separate into input and output
tok_seq = np.array(tok_seq)
X, y = tok_seq[:,:ip_len], tok_seq[:,-op_len:]
#y = KU.to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [26]:
batch_size=32

In [69]:
model = Sequential()
model.add(Embedding(vocab_size, embed_size, embeddings_initializer=Constant(embed_mtx), input_length = seq_length, trainable=False,name = 'MCU_embedding'))
model.add(Bidirectional(LSTM(100,return_sequences=True,go_backwards=True, recurrent_dropout = 0.3),name='BiDi_LSTM_1'))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
#model.add(Bidirectional(LSTM(100,go_backwards=True, recurrent_dropout = 0.3,return_sequences=True), name = 'Bidi_LSTM_2'))
#model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(BatchNormalization())
model.add(Dense(100,name='FC_1'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(100,name='FC_2'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(rate=0.3))
model.add(Flatten())
model.add(Dense(vocab_size, activation='softmax',name='output_layer'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
MCU_embedding (Embedding)    (None, 15, 100)           71000     
_________________________________________________________________
BiDi_LSTM_1 (Bidirectional)  (None, 15, 200)           160800    
_________________________________________________________________
seq_self_attention_1 (SeqSel (None, 15, 200)           12865     
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 200)           800       
_________________________________________________________________
FC_1 (Dense)                 (None, 15, 100)           20100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 100)           400       
_________________________________________________________________
activation_1 (Activation)    (None, 15, 100)           0         
__________

In [70]:
# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

In [71]:
#early_stopper = EarlyStopping(monitor = 'val_acc', patience = 50, mode='min', verbose=1)

In [87]:
# fit model
model.fit(X, y, batch_size=batch_size, epochs=20,verbose=1)

Epoch 1/20
1693/1693 [==============================] - 3s 2ms/step - loss: 0.4000 - sparse_categorical_accuracy: 0.8689
Epoch 2/20
1693/1693 [==============================] - 3s 2ms/step - loss: 0.4296 - sparse_categorical_accuracy: 0.8541
Epoch 3/20
1693/1693 [==============================] - 3s 2ms/step - loss: 0.4489 - sparse_categorical_accuracy: 0.8636
Epoch 4/20
1693/1693 [==============================] - 4s 2ms/step - loss: 0.3971 - sparse_categorical_accuracy: 0.8748
Epoch 5/20
1693/1693 [==============================] - 4s 2ms/step - loss: 0.3554 - sparse_categorical_accuracy: 0.8866
Epoch 6/20
1693/1693 [==============================] - 3s 2ms/step - loss: 0.3643 - sparse_categorical_accuracy: 0.8807A: 0s - loss: 0.3451 - sparse_categorical_accu
Epoch 7/20
1693/1693 [==============================] - 3s 2ms/step - loss: 0.3342 - sparse_categorical_accuracy: 0.8966
Epoch 8/20
1693/1693 [==============================] - 3s 2ms/step - loss: 0.3114 - sparse_categorical_acc

In [88]:
seq_length = len(sequences[0].split()) - 1

In [89]:
# select a seed text
seed_text = sequences[randint(0,len(sequences))]
print(seed_text + '\n')

righthand man and his fathers former partner obadiah stane jeff bridges accepts the award in tonys



In [90]:
encoded = tokenizer.texts_to_sequences([seed_text])[0]
enc = np.array([encoded[:-1]])

In [91]:
# predict probabilities for each word
yhat = model.predict_classes(enc, verbose=1)

1/1 [==============================] - 0s 23ms/step


In [92]:
out_word = ''
for word, index in tokenizer.word_index.items():
    if index == yhat:
        out_word = word
        break

In [93]:
out_word

'tony'

In [94]:
# give credit and reference
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [95]:
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

they tony tony on the that they that so so set of tony tells through them drunk at tony the at tony the him yinsen he missiles that tony tearfully to tearfully to friend of the the behind the terrorist they that that that oceanfront tony intends tony flight soon


In [96]:
sdtx = 'you have full creative freedom to go into infinity and beyond to get the gauntlet'

In [97]:
generated1 = generate_seq(model, tokenizer, seq_length, sdtx, 501)
print(generated1)

functionality functionality of functionality finished of the ii at control of this the soldiers tony soldiers tony the prospect of the suit the cave through through the ii ii through like like like of the soon soon of the at soon soon of soon finished of of the they they soon finished that finished of the the missiles that tony behind that tony tony the terrorist suit thing through through of the so ii the ii ii like like like tonys at tony soldiers that that to quickly the the the prospect oceanfront to they time so of at tony since through like like like that that that tony so version of tony so at finished of the the suit tony thing missiles powered them at that tony tony the thing through of armor tony ii tony the suits cgimapped cgimapped streamlining streamlining streamlining the ii design image that that ii that well conference tony the so tony tony the prospect that the suit they that they both finished finished tony so that tony tony the so that the thing tony missiles missile

In [68]:
#KB.clear_session()

In [ ]:
X.shape

In [ ]:
X

In [ ]:
y.shape

In [ ]:
seq_length